# 01_TGU_validation.ipynb

**Validação do modelo TGU para precessão orbital (Versão Refinada com MASTER TGU)**

Notebook para reproduzir as predições do artigo:
"A Unified Informational Model for Orbital Precession: Spin-Informational Corrections from the TGU Framework"
(Henry Matuchaki, 2025)

Esta versão incorpora o refinamento MASTER TGU com fator de resistência à coerência (ε^{-n}),
garantindo convergência perfeita com GR em regimes de baixa strain.

Objetivos:
- Carregar dados orbitais
- Calcular fator α = 1 + k · (e/a)
- Calcular fator de coerência ε^{-n}
- Aplicar correção refinada na precessão GR
- Comparar com valores do paper e GR
- Visualizar α e convergência em função da tensão informacional (e/a)

In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Importar funções do pacote TGU (original)
from src.tgu_precession import tgu_alpha, tgu_precession, DEFAULT_K

# Importar funções refinadas do MASTER TGU
from src.tgu_master import calculate_coherence_factor, calculate_tgu_precession, RS_INFORMATIONAL, N

print(f"Matuchaki Parameter k = {DEFAULT_K:.6f}")
print(f"Coherence Exponent n = {N}")
print(f"Solar Coherence Radius rs = {RS_INFORMATIONAL:.8f} AU")

## 1. Carregar dados orbitais

In [ ]:
df_orbital = pd.read_csv('../data/orbital_data.csv')
df_orbital

## 2. Calcular predições refinadas TGU para todos os corpos

In [ ]:
results = []

for _, row in df_orbital.iterrows():
    body = row['body']
    e = row['eccentricity_e']
    a = row['semi_major_axis_a_AU']
    delta_gr = row['delta_phi_GR_arcsec_century']
    
    alpha = tgu_alpha(e, a)
    coherence_factor = calculate_coherence_factor(a)
    delta_tgu_refined = calculate_tgu_precession(e, a, delta_gr)
    strain = e / a
    convergence = (delta_tgu_refined / delta_gr) * 100 if delta_gr != 0 else 100.0
    
    results.append({
        'Body': body,
        'e': e,
        'a (AU)': a,
        'Strain (e/a)': round(strain, 4),
        'α': round(alpha, 5),
        'Coherence Factor': round(coherence_factor, 5),
        'Δϕ_GR (arcsec/cent)': delta_gr,
        'Δϕ_TGU Refined (arcsec/cent)': round(delta_tgu_refined, 2),
        'Convergence (%)': round(convergence, 2)
    })

df_results = pd.DataFrame(results)
df_results

## 3. Comparação visual: α e Convergência em função da tensão informacional (e/a)

In [ ]:
fig, ax1 = plt.subplots(figsize=(10, 6))

# Eixo primário: α
ax1.scatter(df_results['Strain (e/a)'], df_results['α'], 
            s=100, c='darkblue', edgecolor='white', linewidth=1.5, label='α')
ax1.set_xlabel('Tensão Informacional (e/a)', fontsize=12)
ax1.set_ylabel('Fator de Correção α', fontsize=12, color='darkblue')
ax1.tick_params(axis='y', labelcolor='darkblue')
ax1.axhline(1.0, color='gray', linestyle='--', alpha=0.7)

# Eixo secundário: Convergência (%)
ax2 = ax1.twinx()
ax2.scatter(df_results['Strain (e/a)'], df_results['Convergence (%)'], 
            s=100, c='darkgreen', edgecolor='white', linewidth=1.5, label='Convergência (%)')
ax2.set_ylabel('Convergência com GR (%)', fontsize=12, color='darkgreen')
ax2.tick_params(axis='y', labelcolor='darkgreen')
ax2.axhline(100.0, color='lightgreen', linestyle='--', alpha=0.7)

# Anotações
for i, row in df_results.iterrows():
    ax1.annotate(row['Body'], (row['Strain (e/a)'], row['α']), 
                 xytext=(5, 5), textcoords='offset points', fontsize=10)

plt.title('Correção TGU Refinada: α e Convergência vs. Tensão Orbital', fontsize=14)
fig.legend(loc='upper right', bbox_to_anchor=(0.9, 0.85))
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 4. Observações finais

- Com o refinamento MASTER TGU (fator de resistência à coerência), o modelo converge perfeitamente com GR em baixa strain (ex.: Mercúrio 100%, Marte 100.24%).
- Para órbitas quase circulares (baixa strain): α ≈ 1 e fator de coerência ≈ 1 → TGU = GR.
- Em alta strain (ex.: exoplanetas), o ganho α domina, oferecendo predições únicas.
- Próximos passos: integrar simulações de spin networks (02_k_derivation_spin.ipynb)
  e testar com mais exoplanetas ou pulsares binários.